In [53]:
import sys
print(sys.executable)

c:\pylab_env\env\Scripts\python.exe


In [54]:
import pandas as pd
import PyQt6
import math

In [55]:
df = pd.read_csv("C:\\pylabrobot_scripts\\test_cherry_pick.csv")
print(df.head())

# get how many unique source plates and destination plates there are
source_plates = df['source_barcode'].unique
destination_plates = df['destination_barcode'].unique

#get the total amount of transfers, for a cherrypick this is the amount of tips to allocate
tip_count = len(df.index)

#calc the number of tip racks this will require, a tip rack holds 96 tips
rack_count = math.ceil((tip_count/96))

#calc the number of of tip carriers, you can fit five tip racks on a carrier
tip_carrrier_count = math.ceil((rack_count/5))


  sample source_barcode soure_well destination_barcode destination_well  \
0  test1           bar1         A1                bar4               A1   
1  test2           bar2         A1                bar4               B1   
2  test3           bar3         A1                bar4               C1   
3  test4           bar1         B1                bar4               D1   
4  test5           bar2         B1                bar4               E1   

   transfer_volume  
0                1  
1                2  
2               20  
3               60  
4              200  


In [56]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend
from pylabrobot.visualizer.visualizer import Visualizer

In [57]:
from pylabrobot.resources.hamilton import STARLetDeck

In [58]:
lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=STARLetDeck())

In [59]:
await lh.setup()

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource teaching_carrier was assigned to the liquid handler.


In [60]:
vis = Visualizer(resource=lh)
await vis.setup()

Websocket server started at http://127.0.0.1:2126
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


In [61]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cor_96_wellplate_360ul_Fb,
    HTF
)

In [62]:
tip_car= TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = tip_rack1 = HTF(name='tips_01', with_tips=False)
tip_car[1] = tip_rack2 = HTF(name='tips_02', with_tips=False)
tip_car[2] = tip_rack3 = HTF(name='tips_03', with_tips=False)
tip_car[3] = tip_rack4 = HTF(name='tips_04', with_tips=False)
tip_car[4] = tip_rack5 = HTF(name='tips_05', with_tips=False)

In [63]:
lh.deck.assign_child_resource(tip_car, rails=10)

Resource tip carrier was assigned to the liquid handler.


In [64]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = plate_1 = Cor_96_wellplate_360ul_Fb(name='plate_01')
plt_car[1] = plate_2 = Cor_96_wellplate_360ul_Fb(name='plate_02')
plt_car[2] = plate_3 = Cor_96_wellplate_360ul_Fb(name='plate_03')

In [65]:
lh.deck.assign_child_resource(plt_car, rails=1)

Resource plate carrier was assigned to the liquid handler.


In [66]:
tip_rack1.fill()

In [67]:
plate_1_liquids = [[(None, 500)]]*96
plate_1.set_well_liquids(plate_1_liquids)

In [68]:
plate_2_liquids = [[(None, 100)], [(None, 500)]]*(96//2)
plate_2.set_well_liquids(plate_2_liquids)